## Transformação camada prata: CountryRegion

In [0]:
%run ../Config/DeltaFunctions

In [0]:
%run ../Config/LogProcessamento 

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType
from pyspark.sql import functions as F
from pyspark.sql import DataFrame, Window

In [0]:
# Habilitar a evolução automática de esquemas
spark.sql("SET spark.databricks.delta.schema.autoMerge.enabled = true")


# Informações da Tabela Fonte
source_table = "person_emailaddress"
source_database = "adventure_works_bronze"
bronze_source_table = spark.read.table(f"{source_database}.{source_table}")

# Informações da Tabela Destino (target)
target_table_name = "person_emailaddress"
target_database = "adventure_works_silver"
target_table = f"{target_database}.{target_table_name}"

primary_keys = ["EmailAddressID"]

In [0]:
expected_schema = StructType([
    StructField("BusinessEntityID", IntegerType(), False),      # int NOT NULL
    StructField("EmailAddressID", IntegerType(), False),        # int IDENTITY(1,1) NOT NULL
    StructField("EmailAddress", StringType(), True),            # nvarchar(50) NULL
    StructField("rowguid", StringType(), False),                # uniqueidentifier NOT NULL
    StructField("ModifiedDate", TimestampType(), False)         # datetime NOT NULL
])


In [0]:
def transform_Person_EmailAddress(EmailAddress: DataFrame) -> DataFrame:
    '''
    Transformação da tabela: Person_EmailAddress

    Parâmetros:
        EmailAddress (DataFrame): DataFrame contendo os dados da tabela Person_EmailAddress

    Retorna:
        DataFrame: O DataFrame resultante após a transformação e deduplicação.
    '''
    
    EmailAddress = EmailAddress.withColumn(
        'rowguid',
        F.when(F.col('rowguid').isNull(), F.expr('uuid()')).otherwise(F.col('rowguid'))
    )
    
    EmailAddress = EmailAddress.withColumn(
        'ModifiedDate',
        F.when(F.col('ModifiedDate').isNull(), F.current_timestamp()).otherwise(F.col('ModifiedDate'))
    )
    
    # Define a função de janela para deduplicar com base nas chaves primárias
    window_spec = Window.partitionBy('BusinessEntityID', 'EmailAddressID').orderBy(F.col('ModifiedDate').desc())
    EmailAddress = EmailAddress.withColumn('row_num', F.row_number().over(window_spec))

    # Filtra para manter apenas a primeira linha em cada partição (sem duplicatas)
    EmailAddress = EmailAddress.filter(F.col('row_num') == 1).drop('row_num')

    # Seleção final com CAST explícito dos tipos de dados
    EmailAddress = EmailAddress.select(
        F.col('BusinessEntityID').cast(IntegerType()).alias('BusinessEntityID'),
        F.col('EmailAddressID').cast(IntegerType()).alias('EmailAddressID'),
        F.col('EmailAddress').cast(StringType()).alias('EmailAddress'),
        F.col('rowguid').cast(StringType()).alias('rowguid'),
        F.col('ModifiedDate').cast(TimestampType()).alias('ModifiedDate')
    )

    return EmailAddress


In [0]:
# Estrutura do log para registrar informações sobre o processo
log_data = {
    "log_tabela": source_table,
    "log_camada": "Silver",
    "log_origem": "adventure_works_bronze",
    "log_destino": "adventure_works_silver",
}

# Registra o início do processo
addlog(**log_data, log_status='Início', atualizacao=0)

try:
    # Realiza a transformação dos dados
    transformed_df = transform_Person_EmailAddress(EmailAddress=bronze_source_table)

    # Verifica rapidamente o número de linhas e o schema do DataFrame
    row_count = transformed_df.count()
    transformed_df.printSchema()

    # Validação do schema
    is_schema_valid = _validate_schema(transformed_df, expected_schema)
    if is_schema_valid:
        addlog(**log_data, log_status='Sucesso', atualizacao=1)
        print("O schema do DataFrame está correto.")
    else:
        raise ValueError("Schema validation failed.")
    
except Exception as e:
    # Registra erro caso ocorra uma exceção
    addlog(**log_data, log_status='Falha', atualizacao=1)
    print(f"Erro ao processar a tabela: {str(e)}")
    raise  

# Se o schema for válido, realiza o upsert
_upsert_silver_table(transformed_df, target_table, primary_keys, not_matched_by_source_action="DELETE")
